In [1]:
import VASPPlotter
import VASPInputHandler
import numpy as np

In [ ]:
# DOS Plot for Full Na to Full Mg.
a = VASPPlotter.Plotter("/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/CrTiO4")
a.dos(what_to_plot={'Cr' : {'spins' : ['summed'], 'orbitals' : ['all']}, 'Ti' :
                    
                    {'spins' : ['summed'], 'orbitals' : ['all']}, 'O' : {'spins' : ['summed'], 'orbitals' : ['all']}},
      colors_and_labels = {'Cr-summed-all' : {'color' : 'black', 'label' : 'Cr'}, 'Ti-summed-all' : {'color' : 'blue', 'label' : 'Ti'}, 'O-summed-all' : {'color' : 'red', 'label' : 'O'}},
      xticks=(True, np.arange(0, 0.100, 0.05)), yticks=(True, np.arange(-3, 3.01, 1)), doscar="DOSCAR", xlim=(0, 0.1), ylim=(-3, 3), smearing = 0.1, show = True, spgp='pnma')

In [ ]:
a = VASPPlotter.Plotter("/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/")
a.avplotter('fd3m', 'Mg', 'Na', calstructure = {'Co0.5Sn1.5O4': {'color': 'royalblue'}, 'CrSnO4': {'color': 'darkorange'}, 'CrTiO4': {'color': 'forestgreen'}, 
                                 'Fe0.5Ti1.5O4': {'color': 'dimgray'}, 'MnSnO4': {'color': 'firebrick'}, 'Ni0.5Sn1.5O4': {'color': 'darkorchid'}})

In [60]:
# Designate new calculation folder and take desire structure from MP database. Change supercell as you want.
a = VASPInputHandler.VASPSetUp("/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/MnTiO4/NaMnTiO4_pnma_first")
a.structure_from_mp("NaMn2O4", "Pnma", False, True)
a.make_supercell([2,1,1])

'/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/MnTiO4/NaMnTiO4_pnma_first/POSCAR'

In [39]:
# Designate new calculation folder with MP ID and get desired structure from MP database.
a = VASPInputHandler.VASPSetUp("/global/cscratch1/sd/yychoi/JCESR/MgSCN/mp-680579_first")
a.structure_from_id()
a.els_to_amts()

{'Zn': 4, 'C': 8, 'S': 8, 'N': 8}

In [61]:
# Change atoms as you want and find most electrostatically stable structure.
a.change_atoms({'Na' : 'Na+', 'Mn' : {'Mn3+' : 0.5, 'Ti4+' : 0.5}, 'O' : 'O2-'}, 100)

'/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/MnTiO4/NaMnTiO4_pnma_first/POSCAR'

In [40]:
# Change atoms one more time. Doing this twice is assuming rapid ion exchage in the process. And perturb poscar
a.change_atoms({'Zn' : 'Mg2+', 'C' : 'C4+', 'N' : 'N3+', 'S' : 'S2-'}, 100)
a.perturb_poscar(0.01)

Something is wrong


'/global/cscratch1/sd/yychoi/JCESR/MgSCN/mp-680579_first/POSCAR'

In [41]:
# Generate Kpoint density with 1000/atom, INCAR, POTCAR, script
a.kpoints(1000)
a.incar(geometry_opt=True, dos=True, MP=True, write_file=True)
a.potcar(MP=True, machine='cori')
a.copy_script(time='12:00:00')

In [2]:
a = VASPInputHandler.VASPBasicAnalysis('/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/Co0.5Sn1.5O4/MgCo0.5Sn1.5O4_fd3m_first')
a.second_run()

calcuation is not converged


nan

In [7]:
b = VASPInputHandler.VASPBasicAnalysis('/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/MnTiO4/MnTiO4_pnma_second/')
b.ehullmp()

0.057252898333333135

In [5]:
import re
i = '/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/MnTiO4/MnTiO4_pnma_second'
re.findall('[A-Z][^A-Z]*', i.split('/')[-1].split('_')[0])[0]

'Mn'

In [5]:
b = VASPPlotter.Plotter('/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels/')

In [ ]:
b.ehullplotter('pnma', ionlist = ['Na', 'Mg'])

In [5]:
import os
import subprocess

In [ ]:
# For generating Secondrun file under MgPostSpinels dictionary
caldir = []
structurelist = [x[0] for x in os.walk('/global/cscratch1/sd/yychoi/JCESR/MgPostSpinels')]
for path in structurelist:
    if path.split('/')[-2] == 'MgPostSpinels' and path.split('/')[-1][-1] == '4':
        caldir.append(path)
        
pathlist = []
for dirs in caldir:
    structurelist = [x[0] for x in os.walk(dirs)]
    for path in structurelist:
        if path.split('/')[-1].split('_')[-1] == 'first' and path.split('/')[-1].split('_')[-2] == 'fd3m':
            a = VASPInputHandler.VASPBasicAnalysis(path)
            a.second_run()
            print(path)
        elif path.split('/')[-1].split('_')[-1] == 'second' and path.split('/')[-1].split('_')[-2] == 'fd3m':
            pathlist.append(path)

In [ ]:
for i, j in enumerate(pathlist):
    # subprocess.call(["ls -alt"], shell=True, cwd=j)
    subprocess.call(["sbatch script"], shell=True, cwd=j)  # for python 2
    print(j)
    print(str(i) + "th job is launched")